In [157]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Ensemble (voting)

In [158]:
from si.data import Dataset
from si.util import summary
from si.util.cv import CrossValidationScore
import os

In [159]:
DIR = os.path.dirname(os.path.realpath('.'))
filename = os.path.join(DIR, 'datasets/breast-bin.data')
dataset = Dataset.from_data(filename)
summary(dataset)

,mean,std,var,max,min
A,4.417740,2.813726,7.917053,10.0,1.0
B,3.134478,3.049276,9.298082,10.0,1.0
C,3.207439,2.969786,8.819630,10.0,1.0
D,2.806867,2.853336,8.141527,10.0,1.0
E,3.216023,2.212715,4.896110,10.0,1.0
F,3.486409,3.619337,13.099601,10.0,1.0
G,3.437768,2.436619,5.937114,10.0,1.0
H,2.866953,3.051449,9.311340,10.0,1.0
I,1.589413,1.713851,2.937284,10.0,1.0
y,0.344778,0.475296,0.225906,1.0,0.0


In [160]:
dataset.toDataframe()

,A,B,C,D,E,F,G,H,I,y
0,8.0,4.0,4.0,1.0,2.0,9.0,3.0,3.0,1.0,1.0
1,5.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,0.0
2,2.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,0.0
3,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,0.0
4,9.0,5.0,8.0,1.0,2.0,3.0,2.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
694,1.0,1.0,1.0,1.0,4.0,3.0,1.0,1.0,1.0,0.0
695,5.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0
696,5.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0
697,2.0,3.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,0.0


In [161]:
# Use accuracy as scorring function
from si.util.metrics import accuracy

### Decision Tree

In [162]:
from si.supervised import DecisionTree

In [163]:
dt = DecisionTree()

In [164]:
cv = CrossValidationScore(dt,dataset,score=accuracy)
cv.run()
cv.toDataframe()

array([[0.9391771 , 0.95348837, 0.95527728],
       [0.92142857, 0.92857143, 0.95714286]])

### Logistic regression

In [165]:
from si.supervised import LogisticRegression
logreg = LogisticRegression()


In [166]:
cv = CrossValidationScore(logreg, dataset, score=accuracy)
cv.run()
cv.toDataframe()

array([[0.96958855, 0.96779964, 0.97137746],
       [0.95714286, 0.95714286, 0.95714286]])

### KNN

In [167]:
from si.supervised import KNN

In [168]:
knn = KNN(7)

In [169]:
cv = CrossValidationScore(knn,dataset,score=accuracy)
cv.run()
cv.toDataframe()

array([[0.96958855, 0.97495528, 0.97853309],
       [0.97857143, 0.96428571, 0.97857143]])

## Ensemble


In [170]:
from si.supervised import Ensemble,majority
en = Ensemble([dt,logreg,knn], majority, accuracy)

In [171]:
cv = CrossValidationScore(en,dataset,score=accuracy)
cv.run()
cv.toDataframe()

array([[0.96779964, 0.980322  , 0.97495528],
       [0.99285714, 0.97142857, 0.96428571]])

## GridSearchCV


In [172]:
from si.util.cv import GridSearchCV

In [173]:
knn = KNN(2)
gscv = GridSearchCV(knn, dataset, parameters={"k":[2,3,4]})
gscv.run()
gscv.toDataframe()